In [36]:
import selenium, requests, time, os, re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm.notebook import tqdm
from pyclowderext import pyclowderext

In [7]:
# Read in the COMPTOX DSSTox/CAS key.
dsst_to_cas = pd.read_csv('DSSTox_Identifiers_and_CASRN_2021r1.csv')

# Read in the Clowder log
log = pd.read_excel('cal_oehha_upload_clowder_log.xlsx')
log['File Name'] = ''
log['URL'] = ''
log['DOI'] = ''
log['HERO ID'] = ''
log['Source'] = 'CAL_OEHHA'
log['Year'] = ''
log['Author'] = 'California Environmental Protection Agency'
log['Study Name'] = ''
log['DSSTox Structure ID'] = ''
log['CASRN'] = ''

userid = '<your_clowder_user_id>'
apikey = '<your_clwoder_api_key_here>'

In [9]:
def t():
    time.sleep(0.25)

In [32]:
Options = webdriver.ChromeOptions()
Options.headless = True 
s=Service(r'C:\Users\JWILLI29\OneDrive - Environmental Protection Agency (EPA)\Profile\Downloads\chromedriver (2).exe')

for i in tqdm(range(len(log))):
    # don't bother re-doing something that's already been done!
    if log['CASRN'].iloc[i] != '':
        continue
    
    filename = log['subDir2'].iloc[i]
    log['File Name'].iloc[i] = filename
    
    chemical_name = filename.split('_')[2]
    
    url = f'https://oehha.ca.gov/chemicals/{chemical_name}'
    log['URL'].iloc[i] = url
    
    # pull up the OEHHA chemical page
    driver = webdriver.Chrome(service=s, options = Options)
    driver.get(url)
    t()
    
    try:
        log['CASRN'].iloc[i] = driver.find_element(By.XPATH, '//*[@id="block-system-main"]/div/div/div[2]').text.split('\n')[1]
    except:
        log['CASRN'].iloc[i] = ''
    t()
    
    driver.quit()

  0%|          | 0/2248 [00:00<?, ?it/s]

In [77]:
log.to_excel('cal_oehha_log_with_metadata.xlsx')

In [61]:
for i in tqdm(range(len(log))):
    try:
        cas = log['CASRN'].iloc[i].replace(' ', '-')
        cas_index = dsst_to_cas['casrn'].tolist().index(cas)
        log['DSSTox Structure ID'].iloc[i] = dsst_to_cas['dtxsid'].iloc[cas_index]
    except ValueError:
        log['DSSTox Structure ID'].iloc[i] = ''
    if log['CASRN'].iloc[i] == '[missing]':
        log['CASRN'].iloc[i] = ''

  0%|          | 0/2248 [00:00<?, ?it/s]

In [68]:
uuid = pd.read_excel('cal_oehha_upload_clowder_log.xlsx')['uuid']
log['uuid'] = uuid

In [76]:
metadata = log[['uuid', 'File Name', 'URL', 'Source', 'Author', 'DSSTox Structure ID', 'CASRN']]
pyclowderext.upload_file_metadata(metadata = metadata, 
                                  idCol = 'uuid', 
                                  userID = userid, 
                                  baseurl = 'https://clowder.edap-cluster.com',
                                  apiKey = apikey)

...pushing metadata to files...


In [82]:
log = log.rename(columns = {"uuid": "clowder_id", "File Name": "document_name"})
log.to_excel('source_cal_oehha_document_map.xlsx', index = False)